## Imports

In [4]:
import mlflow
from mlflow import MlflowClient
from mlflow.types.schema import Schema, ColSpec
from mlflow.types import ParamSchema, ParamSpec
from mlflow.models import ModelSignature
from transformers import pipeline
import torch
import json
import os

2025-02-11 20:28:27.468060: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-11 20:28:27.619652: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739305707.672923     414 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739305707.689308     414 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-11 20:28:27.827642: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## Model

In this part of the code, we load a Transformer model saved on Hugging Face to use it locally (in a pipeline object). This pipeline is then tested with a simple sample.

In [5]:
model_name = "morgana-rodrigues/bert_qa"

qa_pipeline = pipeline(
    'question-answering',
    model=model_name,
    device=0 # -1 means running on CPU
)

config.json:   0%|          | 0.00/582 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


In [6]:
qa_pipeline (context="Take me down to Paradise City where the grass is green and the girls are pretty", question="What colour is the grass")

{'score': 0.9356583952903748, 'start': 49, 'end': 54, 'answer': 'green'}

This class below encapsulates the model in the format that will be logged/registered into MLFlow. It receives a pipeline (or a trainer) as input, saves the model into a temporary folder (called model_name), and log as an artifact into MLFlow. When MLFlow deploys the model, it loads these artifacts into a new pipeline, which can be used to perform inference.

In [7]:
class DistilBERTModel(mlflow.pyfunc.PythonModel):
    def _preprocess(self, inputs):
        context = inputs['context'][0]
        question = inputs['question'][0]
        print("pre processing", context,question)
        return context, question
        
    def load_context(self, context):
        self.model = pipeline(
            'question-answering',
             model=context.artifacts["model"],
             device=0
        )
        
    def predict(self, context, model_input, params):
        in_ctx, question = self._preprocess(model_input)
        output = self.model(context=in_ctx, question=question)
        return output

    @classmethod
    def log_model(cls, model_name, source_trainer = None, source_pipeline = None, demo_folder="demo"): 
        import shutil
        input_schema = Schema(
            [
                ColSpec("string", "context"),
                ColSpec("string", "question"),
            ]
        )
        output_schema = Schema(
            [
                ColSpec("string", "answer")
            ]
        )
        
        params_schema = ParamSchema(
            [
                ParamSpec("show_score", "boolean", False)
            ]
        )
      
        signature = ModelSignature(inputs=input_schema, outputs=output_schema, params=params_schema)
        if source_trainer is not None:
            source_trainer.save_model(model_name)
        elif source_pipeline is not None:
            source_pipeline.save_pretrained(model_name)
             
        requirements = [
            "transformers==4.47.0",
            "tf_keras"
        ]
        mlflow.pyfunc.log_model(
            model_name,
            python_model=cls(),
            artifacts={"model": model_name, "demo": demo_folder},
            signature=signature,
            pip_requirements=requirements
        )
        shutil.rmtree(model_name)

## Model Registry

In [8]:
mlflow.set_experiment(experiment_name='BERT model for Q&A')

<Experiment: artifact_location='/phoenix/mlflow/836489395821734840', creation_time=1739305615466, experiment_id='836489395821734840', last_update_time=1739305615466, lifecycle_stage='active', name='BERT model for Q&A', tags={}>

In [9]:
with mlflow.start_run(run_name='BERT_QA') as run:
    print(f"Run's Artifact URI: {run.info.artifact_uri}")
    DistilBERTModel.log_model(model_name='BERT_QA', source_pipeline=qa_pipeline)
    mlflow.register_model(model_uri = f"runs:/{run.info.run_id}/BERT_QA", name='BERT_QA')

Run's Artifact URI: /phoenix/mlflow/836489395821734840/5fcb6ea0308c4e5a8a8642817af68f56/artifacts


2025/02/11 20:30:13 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - transformers (current: 4.48.3, required: transformers==4.47.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
Successfully registered model 'BERT_QA'.
Created version '1' of model 'BERT_QA'.


## Testing latest model registred

In [9]:
client = mlflow.MlflowClient()
model_metadata = client.get_latest_versions("BERT_QA", stages=["None"])
latest_model_version = model_metadata[0].version
print(latest_model_version, mlflow.models.get_model_info(f"models:/BERT_QA/{latest_model_version}").signature)

2 inputs: 
  ['context': string, 'question': string]
outputs: 
  ['answer': string]
params: 
  ['show_score': boolean (default: False)]



In [10]:
model = mlflow.pyfunc.load_model(model_uri=f"models:/BERT_QA/{latest_model_version}")
context = "Marta is mother of John and Amanda"
question = "what is the name of Marta's daugther?"
model.predict({"context": [context], "question":[question]})

2024/03/22 19:43:23 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - transformers (current: 4.39.1, required: transformers==4.37.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


pre processing ['Marta is mother of John and Amanda'] ["what is the name of Marta's daugther?"]


{'score': 0.6202739477157593, 'start': 28, 'end': 34, 'answer': 'Amanda'}